In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100)
decision_tree = clf_gini.fit(x_train, y_train)
dtfeatures = list(zip(headers, decision_tree.feature_importances_))



In [ ]:
sorted_features_dt = sorted(dtfeatures, key = lambda x:x[1], reverse = True)
print(sorted_features_dt)

In [ ]:
select = SelectFromModel(decision_tree, threshold = 0.1)
select.fit(x_train, y_train)
predictorsdt = []
for feature_list_index in select.get_support(indices=True):
    predictorsdt.append(headers[feature_list_index])
print(predictorsdt)
print(len(predictorsdt))

In [ ]:
to_drop_dt = list(set(headers) - set(predictorsdt))
dt_data = data.drop(to_drop_dt, 1)
dt_corr = dt_data.corr()
figdt = sns.heatmap(dt_corr, 
        annot = True,
        xticklabels=dt_corr.columns,
        yticklabels=dt_corr.columns)
plt.show()

In [ ]:
x_important_train_dt= select.transform(x_train)
x_important_test_dt = select.transform(x_test)
x_important_dt = select.transform(x)
dt = LogisticRegression(C=1e50)
dt.fit(x_important_train_dt, y_train)
print(dt.coef_)

In [ ]:
dt_predicted = dt.predict(x_important_test_dt)
dt_probs = dt.predict_proba(x_important_test_dt)
dt_probs = np.delete(dt_probs, np.s_[0:1], axis =1).flatten()
plt.hist(dt_probs)
plt.show()

In [ ]:
print(metrics.accuracy_score(y_test, dt_predicted))
print(metrics.roc_auc_score(y_test, dt_probs))
print(metrics.confusion_matrix(y_test, dt_predicted))
print(metrics.classification_report(y_test, dt_predicted))

In [ ]:
acc_dt = cross_val_score(LogisticRegression(C = 1e50), x_important_dt, y, scoring = 'accuracy', cv = 10)
print('Accuracy:' + str(acc_dt.mean()))
recall_dt = cross_val_score(LogisticRegression(C = 1e50), x_important_dt, y, scoring = 'recall', cv = 10)
print('Sensitivity:' + str(recall_dt.mean()))
prec_dt = cross_val_score(LogisticRegression(C = 1e50), x_important_dt, y, scoring = specificity, cv = 10)
print('Specificity:' + str(prec_dt.mean()))